In [25]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [26]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

In [27]:
items = pd.read_parquet("items.par")
events = pd.read_parquet("events.par")
items.rename(columns={'book_id': 'item_id'}, inplace=True)
als_recommendations = pd.read_parquet("als_recommendations.parquet")

In [28]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx] # ваш код здесь #
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()# ваш код здесь #
# количество пользователей, которые есть и в train, и в test
common_users =  set(users_train) & set(users_test)# ваш код здесь #

print(len(users_train), len(users_test), len(common_users))

428220 123223 120858


In [29]:
import scipy
import sklearn.preprocessing

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])
srt = sorted(events_train['item_id_enc'], reverse=True)
srt[:5]

/tmp/ipykernel_2194/782784623.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_2194/782784623.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])
/tmp/ipykernel_2194/782784623.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

[43304, 43304, 43250, 43232, 43232]

In [30]:
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

In [31]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

100%|██████████| 50/50 [02:58<00:00,  3.57s/it]


Алгоритм онлайн-рекомендаций  

Возьмём простой алгоритм онлайн-рекомендаций, который будет использовать свойство похожести объектов (item2item similarity), и разберём пример его реализации:  
- Есть заранее посчитанный набор похожих объектов, то есть для каждого объекта есть список объектов, которые на него похожи (в контексте поведения пользователей).  
- Для онлайн-взаимодействия пользователя с каким-либо объектом можно использовать список похожих на него объектов.  
- Теперь можно рекомендовать какие-то объекты (например, несколько первых) из списка. Это будут рекомендации, учитывающие последнее действие пользователя.  

Можно считать, что в таком алгоритме реализован простой счётчик, указывающий на объекты, с которыми пользователь взаимодействовал в последний раз. Feature Store же отвечает за хранение в явном виде статичных признаков объекта — набора похожих на него айтемов. Сейчас вы пошагово реализуете данный алгоритм.  

Шаг 1. Набор похожих объектов
Чтобы получить набор похожих объектов, можно воспользоваться уже известным алгоритмом ALS из библиотеки implicit,  
у которого на такой случай есть удобный метод similar_items (подробнее о нём вы можете прочитать в официальной документации).  
Воспользуемся им и получим по 10 самых похожих айтемов.  

Задание 1 из 6  
Дополните код ниже, чтобы получить набор похожих объектов в similar_items.  
Вы можете подглядеть решение в уроке «Коллаборативная фильтрация: ALS» — там вы реализовывали похожую логику для получения персональных рекомендаций.  

In [32]:
# получим энкодированные идентификаторы всех объектов, известных нам из events_train
train_item_ids_enc = events_train['item_id_enc'].unique()

max_similar_items = 10

# получаем списки похожих объектов, используя ранее полученную ALS-модель
# метод similar_items возвращает и сам объект, как наиболее похожий
# этот объект мы позже отфильтруем, но сейчас запросим на 1 больше
similar_items = als_model.similar_items(train_item_ids_enc, N=max_similar_items+1)


print(len(train_item_ids_enc))
print(len(similar_items[0]))

41474
41474


In [33]:

# преобразуем полученные списки в табличный формат
sim_item_item_ids_enc = similar_items[0]
sim_item_scores = similar_items[1]

similar_items = pd.DataFrame({
    "item_id_enc": train_item_ids_enc,
    "sim_item_id_enc": sim_item_item_ids_enc.tolist(), 
    "score": sim_item_scores.tolist()
    })

similar_items.head()

,item_id_enc,sim_item_id_enc,score
0,2460,"[2460, 2458, 806, 2459, 12528, 1147, 7852, 618...","[0.9999999403953552, 0.9224898815155029, 0.874..."
1,38691,"[38691, 39575, 40111, 25112, 32177, 34430, 367...","[1.0000001192092896, 0.9343445897102356, 0.930..."
2,38867,"[38867, 38023, 38951, 5992, 3865, 10539, 28584...","[1.0, 0.9388757348060608, 0.9345316886901855, ..."
3,39109,"[39109, 37674, 39384, 40645, 17054, 36002, 394...","[0.9999998211860657, 0.9593728184700012, 0.947..."
4,35638,"[35638, 37837, 41337, 39997, 31205, 25389, 324...","[1.0000001192092896, 0.9470844268798828, 0.944..."


In [34]:
similar_items["sim_item_id_enc"].iloc[0]

[2460, 2458, 806, 2459, 12528, 1147, 7852, 618, 6528, 231, 9519]

In [35]:
similar_items = similar_items.explode(["sim_item_id_enc", "score"], ignore_index=True)

similar_items.head()

,item_id_enc,sim_item_id_enc,score
0,2460,2460,1.0
1,2460,2458,0.92249
2,2460,806,0.874765
3,2460,2459,0.873763
4,2460,12528,0.850654


In [36]:
# приводим типы данных
similar_items["sim_item_id_enc"] = similar_items["sim_item_id_enc"].astype("int")
similar_items["score"] = similar_items["score"].astype("float")

# получаем изначальные идентификаторы
similar_items["item_id_1"] = item_encoder.inverse_transform(similar_items["item_id_enc"])
similar_items["item_id_2"] = item_encoder.inverse_transform(similar_items["sim_item_id_enc"])

similar_items = similar_items.drop(columns=["item_id_enc", "sim_item_id_enc"])

# убираем пары с одинаковыми объектами
similar_items = similar_items.query("item_id_1 != item_id_2")

similar_items.head()

,score,item_id_1,item_id_2
1,0.922490,22034,22026
2,0.874765,22034,6882
3,0.873763,22034,22028
4,0.850654,22034,364089
5,0.835730,22034,9827


In [37]:
# Фильтруем похожие объекты для item_id = 7126 и находим самый похожий (с максимальным score)
most_similar_item = similar_items[similar_items["item_id_1"] == 7126] \
    .sort_values(by="score", ascending=False) \
    .iloc[0]["item_id_2"]

print(f"Наиболее похожий объект для item_id 7126: {most_similar_item}")

Наиболее похожий объект для item_id 7126: 7190.0


In [38]:
similar_items[similar_items['item_id_1'] == 7126]

,score,item_id_1,item_id_2
25873,0.948725,7126,7190
25874,0.940997,7126,24280
25875,0.930144,7126,1953
25876,0.925066,7126,58696
25877,0.916340,7126,38296
25878,0.916015,7126,2932
25879,0.913951,7126,7184
25880,0.911433,7126,387749
25881,0.909872,7126,7733
25882,0.909454,7126,30597


In [39]:
# Полученный набор вскоре вам снова понадобится, так что сохраните его в файле: 

similar_items.to_parquet("similar_items.parquet")

Полезно убедиться, что полученный набор действительно содержит похожие данные.  
Например, можно оценить глазами списки похожих объектов для каких-то уже известных.  
Создадим для этой цели функцию print_sim_items.

In [40]:
def print_sim_items(item_id, similar_items):

    item_columns_to_use = ["item_id", "author", "title", "genre_and_votes", "average_rating", "ratings_count"]
    
    item_id_1 = items.query("item_id == @item_id")[item_columns_to_use]
    display(item_id_1)
    
    si = similar_items.query("item_id_1 == @item_id")
    si = si.merge(items[item_columns_to_use].set_index("item_id"), left_on="item_id_2", right_index=True)
    display(si)

In [41]:
print_sim_items(7126, similar_items)

,item_id,author,title,genre_and_votes,average_rating,ratings_count
738212,7126,"Alexandre Dumas, Robin Buss",The Count of Monte Cristo,"{'Classics': 18271, 'Fiction': 6761, 'Historic...",4.22,564778


,score,item_id_1,item_id_2,author,title,genre_and_votes,average_rating,ratings_count
25873,0.948725,7126,7190,Alexandre Dumas,"The Three Musketeers (The D'Artagnan Romances,...","{'Classics': 9823, 'Fiction': 3256, 'Historica...",4.06,198892
25874,0.940997,7126,24280,"Victor Hugo, Lee Fahnestock, Norman MacAfee",Les Misérables,"{'Classics': 15159, 'Fiction': 5061, 'Historic...",4.14,519758
25875,0.930144,7126,1953,"Charles Dickens, Richard Maxwell",A Tale of Two Cities,"{'Classics': 20021, 'Fiction': 6969, 'Historic...",3.82,646983
25876,0.925066,7126,58696,"Charles Dickens, Jeremy Tambling",David Copperfield,"{'Classics': 7795, 'Fiction': 2831, 'Literatur...",3.96,152764
25877,0.916340,7126,38296,James Fenimore Cooper,The Last of the Mohicans (The Leatherstocking ...,"{'Classics': 2791, 'Fiction': 1314, 'Historica...",3.69,68582
25878,0.916015,7126,2932,"Daniel Defoe, Virginia Woolf",Robinson Crusoe,"{'Classics': 7725, 'Fiction': 3305, 'Adventure...",3.66,181415
25879,0.913951,7126,7184,"Alexandre Dumas, David Coward, Auguste Maquet","Twenty Years After (The D'Artagnan Romances, #2)","{'Classics': 933, 'Fiction': 392, 'Historical-...",4.01,14769
25880,0.911433,7126,387749,Lew Wallace,Ben-Hur: A Tale of the Christ,"{'Classics': 884, 'Historical-Historical Ficti...",4.02,25686
25881,0.909872,7126,7733,"Jonathan Swift, Robert DeMaria Jr.",Gulliver's Travels,"{'Classics': 8077, 'Fiction': 3399, 'Fantasy':...",3.56,171663
25882,0.909454,7126,30597,"Victor Hugo, Walter J. Cobb",The Hunchback of Notre-Dame,"{'Classics': 6285, 'Fiction': 1974, 'Historica...",3.97,122720


Попробуйте оценить похожие айтемы для следующих известных книг (числа — идентификаторы item_id):  
- 7144: Ф. М. Достоевский «Преступление и наказание»;  
- 16299: Агата Кристи «Десять негритят»;  
- 3: Джоан Роулинг «Гарри Поттер и философский камень»;  
- 18135: Уильям Шекспир «Ромео и Джульетта»;  
- 17245: Брэм Стокер «Дракула».  
Для этого вызовите функцию, указанную выше, например:

In [42]:
print_sim_items(7144, similar_items)

,item_id,author,title,genre_and_votes,average_rating,ratings_count
1909078,7144,"Fyodor Dostoyevsky, David McDuff, Fyodor Dosto...",Crime and Punishment,"{'Classics': 15812, 'Fiction': 8028, 'Cultural...",4.19,390293


,score,item_id_1,item_id_2,author,title,genre_and_votes,average_rating,ratings_count
66760,0.964479,7144,12505,"Fyodor Dostoyevsky, Anna Brailovsky, Constance...",The Idiot,"{'Classics': 4036, 'Fiction': 2576}",4.18,76392
66761,0.953918,7144,12857,"Fyodor Dostoyevsky, Constance Garnett",The Gambler,"{'Classics': 946, 'Fiction': 729, 'Cultural-Ru...",3.88,22024
66762,0.952009,7144,67326,Fyodor Dostoyevsky,Poor Folk,"{'Classics': 320, 'Fiction': 235, 'Literature-...",3.73,4957
66763,0.946847,7144,5508624,Leo Tolstoy,Family Happiness,"{'Classics': 140, 'Fiction': 112, 'Cultural-Ru...",3.85,3337
66764,0.939762,7144,4934,"Fyodor Dostoyevsky, Fyodor Dostoyevsky, Richar...",The Brothers Karamazov,"{'Classics': 7496, 'Fiction': 5491, 'Cultural-...",4.31,158410
66765,0.938018,7144,17877,"Fyodor Dostoyevsky, Constance Garnett",The House of the Dead,"{'Classics': 533, 'Fiction': 441, 'Cultural-Ru...",4.04,8548
66766,0.937007,7144,929782,"Jack London, Andrew Sinclair",Martin Eden,"{'Classics': 435, 'Fiction': 405, 'Literature-...",4.39,13257
66767,0.936360,7144,28382,Nikolai Gogol,Diary of a Madman and Other Stories,"{'Classics': 284, 'Fiction': 243, 'Short Stori...",4.09,6241
66768,0.936320,7144,17690,"Franz Kafka, Max Brod, Willa Muir, Edwin Muir",The Trial,"{'Classics': 4607, 'Fiction': 4173, 'Literatur...",3.98,135862
66769,0.934541,7144,63038,Victor Hugo,The Man Who Laughs,"{'Classics': 352, 'Fiction': 176, 'Cultural-Fr...",4.22,5449


In [43]:
print_sim_items(16299, similar_items)

,item_id,author,title,genre_and_votes,average_rating,ratings_count
794321,16299,Agatha Christie,And Then There Were None,"{'Mystery': 12703, 'Classics': 6623, 'Fiction'...",4.23,429352


,score,item_id_1,item_id_2,author,title,genre_and_votes,average_rating,ratings_count
2388,0.858777,16299,16328,Agatha Christie,"The Murder of Roger Ackroyd (Hercule Poirot, #4)","{'Mystery': 5069, 'Fiction': 1765, 'Classics':...",4.20,74002
2389,0.817708,16299,16322,Agatha Christie,"The A.B.C. Murders (Hercule Poirot, #13)","{'Mystery': 3635, 'Fiction': 1085, 'Mystery-Cr...",3.98,51072
2390,0.811799,16299,16343,Agatha Christie,The Mysterious Affair at Styles (Hercule Poiro...,"{'Mystery': 5803, 'Fiction': 1826, 'Classics':...",3.98,142922
2391,0.794610,16299,16315,Agatha Christie,Crooked House,"{'Mystery': 1778, 'Fiction': 523, 'Mystery-Cri...",3.98,16312
2392,0.784852,16299,853510,Agatha Christie,"Murder on the Orient Express (Hercule Poirot, ...","{'Mystery': 9992, 'Classics': 4865, 'Fiction':...",4.16,25335
2393,0.781295,16299,131359,Agatha Christie,"Death on the Nile (Hercule Poirot, #17)","{'Mystery': 4103, 'Fiction': 1296, 'Mystery-Cr...",4.07,66646
2394,0.776483,16299,948072,"Charles Osborne, Agatha Christie",The Unexpected Guest,"{'Mystery': 311, 'Fiction': 88, 'Mystery-Crime...",3.99,3108
2395,0.774083,16299,121648,Agatha Christie,"Five Little Pigs (Hercule Poirot, #24)","{'Mystery': 1736, 'Fiction': 514, 'Mystery-Cri...",3.96,20208
2396,0.769652,16299,639787,Agatha Christie,"The Murder on the Links (Hercule Poirot, #2)","{'Mystery': 2341, 'Fiction': 672, 'Mystery-Cri...",3.80,19533
2397,0.761913,16299,16366,Agatha Christie,Endless Night,"{'Mystery': 1032, 'Fiction': 289, 'Mystery-Cri...",3.75,10155


In [44]:
print_sim_items(3, similar_items)

,item_id,author,title,genre_and_votes,average_rating,ratings_count
1584855,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad...",4.45,4765497


,score,item_id_1,item_id_2,author,title,genre_and_votes,average_rating,ratings_count
10297,0.986763,3,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict...",4.38,1821802
10298,0.974947,3,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,"{'Fantasy': 49784, 'Young Adult': 15393, 'Fict...",4.53,1876252
10299,0.954390,3,6,"J.K. Rowling, Mary GrandPré",Harry Potter and the Goblet of Fire (Harry Pot...,"{'Fantasy': 48257, 'Young Adult': 15483, 'Fict...",4.53,1792561
10300,0.934225,3,2,"J.K. Rowling, Mary GrandPré",Harry Potter and the Order of the Phoenix (Har...,"{'Fantasy': 46485, 'Young Adult': 15194, 'Fict...",4.47,1766895
10301,0.922894,3,1,J.K. Rowling,Harry Potter and the Half-Blood Prince (Harry ...,"{'Fantasy': 46400, 'Young Adult': 15083, 'Fict...",4.54,1713866
10302,0.907875,3,136251,J.K. Rowling,Harry Potter and the Deathly Hallows (Harry Po...,"{'Fantasy': 46667, 'Young Adult': 15403, 'Fict...",4.62,1784684
10303,0.861305,3,8388506,"Bruno Nogueira, João Quadros","Tubo de Ensaio, Parte II","{'Humor': 4, 'Humor-Comedy': 1}",3.26,39
10304,0.861305,3,6379485,"Bruno Nogueira, João Quadros",Tubo de Ensaio,"{'Humor': 5, 'Humor-Comedy': 2}",3.27,44
10305,0.838405,3,7904207,Jim Henry,Antiquity Calais: Standing at Armageddon (The ...,None,4.61,16
10306,0.737723,3,8226034,Hans Scherfig,Frydenholm,"{'Historical-Historical Fiction': 3, 'Fiction'...",4.06,98


In [45]:
print_sim_items(18135, similar_items)

,item_id,author,title,genre_and_votes,average_rating,ratings_count
1592356,18135,"William Shakespeare, Paul Werstine, Barbara A....",Romeo and Juliet,"{'Classics': 26113, 'Plays': 9558, 'Fiction': ...",3.74,1656919


,score,item_id_1,item_id_2,author,title,genre_and_votes,average_rating,ratings_count
92951,0.920891,18135,8852,William Shakespeare,Macbeth,"{'Classics': 16116, 'Plays': 8310, 'Fiction': ...",3.88,502298
92952,0.889775,18135,1420,"William Shakespeare, Harold Bloom, Rex Gibson",Hamlet,"{'Classics': 17549, 'Plays': 8817, 'Fiction': ...",4.01,526122
92953,0.888842,18135,7728,"Sophocles, J.E. Thomas","Antigone (The Theban Plays, #3)","{'Classics': 3847, 'Plays': 2667, 'Drama': 897...",3.61,69075
92954,0.880297,18135,17250,"Arthur Miller, Christopher Bigsby",The Crucible,"{'Classics': 7902, 'Plays': 4768, 'Fiction': 2...",3.55,247565
92955,0.876469,18135,1622,"William Shakespeare, Paul Werstine, Barbara A....",A Midsummer Night's Dream,"{'Classics': 12032, 'Plays': 6438, 'Fiction': ...",3.94,340695
92956,0.869903,18135,12296,"Nathaniel Hawthorne, Thomas E. Connolly, Faust...",The Scarlet Letter,"{'Classics': 19456, 'Fiction': 6716}",3.37,515452
92957,0.866114,18135,13006,"William Shakespeare, Roma Gill",Julius Caesar,"{'Classics': 5864, 'Plays': 3637, 'Fiction': 1...",3.66,121890
92958,0.854540,18135,1554,"Sophocles, J.E. Thomas","Oedipus Rex (The Theban Plays, #1)","{'Classics': 4512, 'Plays': 3029, 'Drama': 100...",3.67,122126
92959,0.848521,18135,12996,William Shakespeare,Othello,"{'Classics': 8696, 'Plays': 5397, 'Fiction': 1...",3.88,242511
92960,0.836064,18135,2956,"Mark Twain, Guy Cardwell, John Seelye, Walter ...",The Adventures of Huckleberry Finn,"{'Classics': 20909, 'Fiction': 9269, 'Historic...",3.80,969291


In [46]:
print_sim_items(17245, similar_items)

,item_id,author,title,genre_and_votes,average_rating,ratings_count
1058909,17245,"Bram Stoker, Nina Auerbach, David J. Skal",Dracula,"{'Classics': 19603, 'Horror': 10601, 'Fiction'...",3.98,636895


,score,item_id_1,item_id_2,author,title,genre_and_votes,average_rating,ratings_count
23937,0.928823,17245,480204,"Gaston Leroux, Alexander Teixeira de Mattos",The Phantom of the Opera,"{'Classics': 7010, 'Fiction': 2103, 'Horror': ...",3.97,144859
23938,0.900337,17245,51496,"Robert Louis Stevenson, Vladimir Nabokov, Merv...",The Strange Case of Dr. Jekyll and Mr. Hyde,"{'Classics': 12342, 'Fiction': 4037, 'Horror':...",3.79,229898
23939,0.898938,17245,93261,Washington Irving,The Legend of Sleepy Hollow,"{'Classics': 2594, 'Horror': 1182, 'Fiction': ...",3.74,26776
23940,0.897706,17245,295,Robert Louis Stevenson,Treasure Island,"{'Classics': 11249, 'Fiction': 4405, 'Adventur...",3.82,274424
23941,0.896470,17245,2623,"Charles Dickens, Marisa Sestino",Great Expectations,"{'Classics': 19645, 'Fiction': 6662, 'Literatu...",3.75,468462
23942,0.895993,17245,18254,"Charles Dickens, Philip Horne, Gerald Dickens",Oliver Twist,"{'Classics': 11450, 'Fiction': 3656, 'Historic...",3.85,235560
23943,0.886899,17245,7190,Alexandre Dumas,"The Three Musketeers (The D'Artagnan Romances,...","{'Classics': 9823, 'Fiction': 3256, 'Historica...",4.06,198892
23944,0.881911,17245,24213,"Lewis Carroll, John Tenniel, Martin Gardner",Alice's Adventures in Wonderland & Through the...,"{'Classics': 11568, 'Fantasy': 6184, 'Fiction'...",4.06,344482
23945,0.878392,17245,2932,"Daniel Defoe, Virginia Woolf",Robinson Crusoe,"{'Classics': 7725, 'Fiction': 3305, 'Adventure...",3.66,181415
23946,0.870232,17245,1953,"Charles Dickens, Richard Maxwell",A Tale of Two Cities,"{'Classics': 20021, 'Fiction': 6969, 'Historic...",3.82,646983
